In [2]:
import pandas as pd
import numpy as np
import os.path
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
import cv2
from keras.applications import InceptionV3
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense , Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
Data_csv = '/media/ghost/New Volume1/ML dataset/image_classification/image_list.csv'
Data_path = '/media/ghost/New Volume1/ML dataset/image_classification/images'

In [3]:
df = pd.read_csv(Data_csv)
NUM_CLASSES = 30
IMAGE_X_SIZE = 300
IMAGE_Y_SIZE = 300

In [4]:
df.head()

,Image_id,Animal
0,Img-1252.jpg,beaver
1,Img-89.jpg,chimpanzee
2,Img-21675.jpg,mouse
3,Img-4387.jpg,rhinoceros
4,Img-2105.jpg,antelope


In [5]:
le = LabelEncoder()
le.fit(df['Animal'])
df['Animal_id'] = le.transform(df['Animal'])

In [6]:
le.classes_

array(['antelope', 'bat', 'beaver', 'bobcat', 'buffalo', 'chihuahua',
       'chimpanzee', 'collie', 'dalmatian', 'german+shepherd',
       'grizzly+bear', 'hippopotamus', 'horse', 'killer+whale', 'mole',
       'moose', 'mouse', 'otter', 'ox', 'persian+cat', 'raccoon', 'rat',
       'rhinoceros', 'seal', 'siamese+cat', 'spider+monkey', 'squirrel',
       'walrus', 'weasel', 'wolf'], dtype=object)

In [7]:
df.head()

,Image_id,Animal,Animal_id
0,Img-1252.jpg,beaver,2
1,Img-89.jpg,chimpanzee,6
2,Img-21675.jpg,mouse,16
3,Img-4387.jpg,rhinoceros,22
4,Img-2105.jpg,antelope,0


In [8]:
def load_images(data,image_path,start_from,x_size,y_size,train):
    image_resize = (x_size,y_size)
    x = []
    y = []
    ids = []
    
    if train == True:
        up_to = start_from + 3000
    else:
        up_to = -1
        
    for img_id,Ani,Ani_id in tqdm(data.iloc[start_from:up_to,:].values, miniters = 100):
        path = os.path.join(image_path,img_id)
        img = cv2.imread(path)
        x.append(cv2.resize(img, image_resize))
        y.append(Ani_id)
        ohe = OneHotEncoder(sparse=False)
        he = ohe.fit_transform(np.array(y).reshape(len(y),1))
        
    return x,he

In [9]:
def get_data(data,image_path,start_from,x_size,y_size):
    
    x , y = load_images(data,image_path,start_from,x_size,y_size,True)
    x = np.array(x,np.float32)/255
    

#     return train_test_split(x,y,test_size = .1, random_state = 0)
    return x,y

In [10]:
base_model = InceptionV3(weights = '/media/ghost/New Volume1/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5', input_shape = (IMAGE_X_SIZE,IMAGE_Y_SIZE,3))
for layers in base_model.layers[1:]:
    layers.trainable = False
    
model = Sequential([
    base_model,
    Dense(1024,activation = 'sigmoid'),
    Dropout(0.5),
    Dense(1024,activation = 'sigmoid'),
    Dropout(0.5),
    Dense(512,activation = 'sigmoid'),
    Dropout(0.5),
    Dense(NUM_CLASSES,activation = 'softmax')
])

for layers in model.layers[0].layers[-20:]:
    layers.trainable = True

optimizer = Adam(0.0001,decay = 0.000001)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer , metrics = ['accuracy'])
model.summary()
model.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1000)              23851784  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1025024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
__________

In [38]:
x , y = get_data(df,Data_path,0,IMAGE_X_SIZE,IMAGE_Y_SIZE)
# X_train, X_test, y_train, y_test = get_data(df,Data_path,0,IMAGE_X_SIZE,IMAGE_Y_SIZE)

100%|██████████| 3000/3000 [01:37<00:00, 30.90it/s]


In [14]:
callbacks_list = [
    EarlyStopping(monitor='val_loss',patience=10),
    ModelCheckpoint(filepath = 'resnet50_4000.h5',monitor='val_loss',save_best_only=True)
]

In [12]:
model.load_weights("IInceptionV3_0000_1.h5")

In [39]:
history = model.fit(
    x,
    y,
    epochs = 50,
    validation_split = 0.2,
    batch_size = 64,
    shuffle = True,
#     steps_per_epoch = 80,
#     validation_steps = 10
)

Train on 2400 samples, validate on 600 samples
Epoch 1/50
2400/2400 [==============================] - 164s 68ms/step - loss: 0.6423 - acc: 0.6912 - val_loss: 4.0740 - val_acc: 0.4800
Epoch 2/50
2400/2400 [==============================] - 160s 67ms/step - loss: 0.6394 - acc: 0.6883 - val_loss: 4.0580 - val_acc: 0.4583
Epoch 3/50
2400/2400 [==============================] - 154s 64ms/step - loss: 0.6096 - acc: 0.6917 - val_loss: 3.9291 - val_acc: 0.4717
Epoch 4/50
2400/2400 [==============================] - 154s 64ms/step - loss: 0.6142 - acc: 0.7054 - val_loss: 4.0630 - val_acc: 0.4650
Epoch 5/50
2400/2400 [==============================] - 154s 64ms/step - loss: 0.5903 - acc: 0.7104 - val_loss: 4.0767 - val_acc: 0.4533
Epoch 6/50
2400/2400 [==============================] - 154s 64ms/step - loss: 0.5762 - acc: 0.7075 - val_loss: 4.1230 - val_acc: 0.4550
Epoch 7/50
2400/2400 [==============================] - 154s 64ms/step - loss: 0.5782 - acc: 0.7042 - val_loss: 4.0527 - val_acc: 0

In [40]:
 model.save('IIInceptionV3_0000_1.h5')
